In [30]:
register_path = "/data/students/bigdata-01QYD/Lab6_DBD/register.csv"
stations_path = "/data/students/bigdata-01QYD/Lab6_DBD/stations.csv"
outputPath = "res_out_Lab7/"
treshold = 0.4

In [31]:

registerDF = spark.read.load(register_path,\
                            format="csv",\
                            sep="\t",\
                            header=True,\
                            inferSchema=True)        

In [32]:
registerFiltered = registerDF.filter("used_slots <> 0 OR free_slots <> 0")

In [33]:
registerFiltered.printSchema()

root
 |-- station: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- used_slots: integer (nullable = true)
 |-- free_slots: integer (nullable = true)



In [34]:
#registerWithTimeSlot = registerFiltered.selectExpr("station","date_format(timestamp,'EE') as weekday",\
#"hour(timestamp) as hour","used_slots","free_slots")

In [35]:
#groupedCriticalRegister = registerWithTimeSlot.filter("free_slots = 0").groupBy("station","weekday","hour")\
#.count()

In [36]:
#groupedTotalReadings = registerWithTimeSlot.groupBy("station","weekday","hour")\
#.count().withColumnRenamed("station","station1").withColumnRenamed("weekday","weekday1").withColumnRenamed("hour","hour1").withColumnRenamed("count","countTOT")

In [37]:
#joinedDF = groupedCriticalRegister.join(groupedTotalReadings,\
#                             (groupedCriticalRegister.station == groupedTotalReadings.station1) &\
#                             (groupedCriticalRegister.weekday == groupedTotalReadings.weekday1) &\
#                             (groupedCriticalRegister.hour == groupedTotalReadings.hour1))

In [ ]:
#joinedDF.printSchema()

In [39]:
#spark.udf.register("crit_value", lambda countCrit, countTOT: countCrit/countTOT, "FLOAT")

#stationTS_withCV = joinedDF.selectExpr("station","weekday","hour",\
#                                       "crit_value(count, countTOT) AS crit_value")

In [48]:
#---------------------- ALTERNATIVE SOLUTION ---> AVG
#User Defined Function called full(Integer free_slots)
# that returns 
# -- 1 if free_slots is equal to 0
# -- 0 if free_slots is greater than 0
def fullFunction(free_slots):
    if free_slots==0:
        return 1
    else:
        return 0
    
    
spark.udf.register("full", fullFunction)

stationWeekdayHourDF = registerFiltered.selectExpr("station", "date_format(timestamp,'EE') as dayofweek",\
                                             "hour(timestamp) as hour", "full(free_slots) as fullstatus")
rgdStationWeekdayHourDF = stationWeekdayHourDF.groupBy("station", "dayofweek", "hour")
# The criticality is equal to the average of fullStatus
stationWeekdayHourCriticalityDF = rgdStationWeekdayHourDF.agg({"fullStatus": "avg"})\
.withColumnRenamed("avg(fullStatus)", "criticality")

#------------------------END OF ALTERNATIVE SOL. --------------------------

In [14]:
stationTS_withCV.printSchema()

root
 |-- station: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- crit_value: float (nullable = true)



In [50]:
tresholdFilteringDF = stationWeekdayHourCriticalityDF.filter("criticality > "+str(treshold))

In [51]:
stationsDF = spark.read.load(stations_path,\
                             format="csv",\
                             sep="\t",\
                             header=True,\
                             inferSchema=True
                            )

In [52]:
stationsDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- name: string (nullable = true)



In [53]:
joinedDF = tresholdFilteringDF.join(stationsDF,\
                     stationTS_withCV.station==stationsDF.id)

In [54]:
joinedDF.printSchema()

root
 |-- station: integer (nullable = true)
 |-- dayofweek: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- criticality: double (nullable = true)
 |-- id: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- name: string (nullable = true)



In [59]:
dfToStore = joinedDF.select("station","dayofweek","hour","criticality",\
                            "longitude","latitude")\
.sort(joinedDF.criticality.desc(), joinedDF.station, joinedDF.dayofweek, joinedDF.hour)

In [60]:
dfToStore.coalesce(1).write.csv(outputPath, header=True)